<a href="https://colab.research.google.com/github/deivismartinez/NER-Medical-Uninorte/blob/main/Copia_de_anotaciones_metricas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cd /content && git clone https://github.com/nlplab/brat.git

In [ ]:
!wget https://github.com/deivismartinez/NER-Medical-Uninorte/raw/main/systemd.zip?download=1 -O system.zip

In [ ]:
!unzip system.zip -d /content/systemd/

In [ ]:
def change_tags(old_value, new_value, file_name):
  with open(file_name, "rt") as file:
    x = file.read()
	
  with open(file_name, "wt") as file:
    x = x.replace(old_value,new_value)
    file.write(x)

In [ ]:
def do_changes():
  gold_files = glob.glob("/content/systemd/systemd/gold/*.ann")
  for file in gold_files:
    change(file)

  for model_name in glob.glob("/content/systemd/systemd/*"):
    if "model_" in model_name:
      gold_files = glob.glob(f"{model_name}/test2/*.ann")
      for file in gold_files:
        change(file)

In [ ]:
def change(file_name):
  change_tags(old_value = "NOMBRE_PERSONAL_SANITARIO", new_value="NOMBRE",file_name=file_name)
  change_tags(old_value = "NOMBRE_SUJETO_ASISTENCIA", new_value="NOMBRE",file_name=file_name)
  change_tags(old_value = "FAMILIARES_SUJETO_ASISTENCIA", new_value="NOMBRE",file_name=file_name)
  change_tags(old_value = "OTROS_SUJETO_ASISTENCIA", new_value="NOMBRE",file_name=file_name)

  change_tags(old_value = "TERRITORIO", new_value="LUGAR",file_name=file_name)
  change_tags(old_value = "PAIS", new_value="LUGAR",file_name=file_name)
  change_tags(old_value = "CALLE", new_value="LUGAR",file_name=file_name)

  change_tags(old_value = "CENTRO_SALUD", new_value="ORGANIZACION",file_name=file_name)
  change_tags(old_value = "HOSPITAL", new_value="ORGANIZACION",file_name=file_name)
  change_tags(old_value = "INSTITUCION", new_value="ORGANIZACION",file_name=file_name)

In [ ]:
do_changes()

In [ ]:
def delete_tags(file_name):
  with open(file_name, "rt") as file:
    x = [row for row in file.readlines() if "NOMBRE" in row]
  with open(file_name, "wt") as file:
    file.writelines(x)

In [ ]:
!pip install seqeval

In [ ]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics.sequence_labeling import precision_score, recall_score
from seqeval.metrics import f1_score
import glob
import os
from typing import List
import zipfile
from google.colab import files

In [ ]:
def do_file_gold_conll():
  gold_files = glob.glob("/content/systemd/systemd/gold/*.txt")

  for file in gold_files:
    #print(file[:-3]+"ann")
    delete_tags(file[:-3]+"ann")
    command = f"""
    python /content/brat/tools/anntoconll.py "{file}"
    """
    os.system(command)

In [ ]:
def do_file_pred_conll(model):
  pred_files = glob.glob(f"{model}/test2/*.txt")

  for file in pred_files:
    delete_tags(file[:-3]+"ann")
    command = f"""
    python /content/brat/tools/anntoconll.py "{file}"
    """
    os.system(command)

In [ ]:
def create_file():
  do_file_gold_conll()
  for model_name in glob.glob("/content/systemd/systemd/*"):
    if "model_" in model_name:
      do_file_pred_conll(model=model_name)

In [ ]:
create_file()

In [ ]:
models_name = {"/content/systemd/systemd/model_1":"roberta-large-bne-capitel-ner",
"/content/systemd/systemd/model_2":"roberta-base-bne-capitel-ner-plus",
"/content/systemd/systemd/model_3":"roberta-base-bne-capitel-ner",
"/content/systemd/systemd/model_4":"bsc-bio-ehr-es-pharmaconer",
"/content/systemd/systemd/model_5":"roberta-base-bne-capitel-pos", 
"/content/systemd/systemd/model_6":"bert-base-multilingual-cased-ner-hrl", 
"/content/systemd/systemd/model_7":"xlm-roberta-base-wikiann-ner", 
"/content/systemd/systemd/model_8":"xlm-roberta-base-ner-hrl", 
"/content/systemd/systemd/model_9":"distilbert-base-multilingual-cased-ner-hrl",
"/content/systemd/systemd/model_10":"xlm-roberta-large-ner-hrl", 
"/content/systemd/systemd/model_11":"bert-spanish-cased-finetuned-ner", 
"/content/systemd/systemd/model_12":"TinyBERT-spanish-uncased-finetuned-ner", 
"/content/systemd/systemd/model_13":"RuPERTa-base-finetuned-ner", 
"/content/systemd/systemd/model_14":"ner-spanish-large", 
"/content/systemd/systemd/model_15":"ner-multi", 
"/content/systemd/systemd/model_16":"anglicisms-spanish-mbert", 
"/content/systemd/systemd/model_17":"anglicisms-spanish-flair-cs", 
"/content/systemd/systemd/model_18":"xlm-roberta-large-ner-spanish", 
"/content/systemd/systemd/model_19":"wikineural-multilingual-ner", 
"/content/systemd/systemd/model_20":"distilbert-base-multilingual-cased-finetuned-conll2003-ner", 
"/content/systemd/systemd/model_21":"bertin-base-ner-conll2002-es", 
"/content/systemd/systemd/model_22":"xlm-roberta-base-finetuned-panx-ner",
"/content/systemd/systemd/model_23":"stanza-es"}

In [ ]:
def read_tags(fpath: str) -> List:
  lines = open(fpath, 'r').read()
  lines = lines.split("\n")
  lines = [line.split("\t") for line in lines]
  tags = [line[0] for line in lines if len(line) > 1]
  return tags

In [ ]:
def do_table(model,file_t,file_tp):
  gold_files = glob.glob("/content/systemd/systemd/gold/*.conll")
  print(gold_files)
  pred_files = glob.glob(f"{model}/test2/*.conll")
  gold_tags = [read_tags(file) for file in gold_files]
  pred_tags = [read_tags(file) for file in pred_files]
  f1 = f1_score(y_true=gold_tags, y_pred=pred_tags)
  print(f1)
  precision = precision_score(y_true=gold_tags, y_pred=pred_tags)
  print(precision)
  recall = recall_score(y_true=gold_tags, y_pred=pred_tags)
  print(recall)
  file_tp.write(models_name[model]+'\t'+str(precision)+'\t'+str(recall)+'\t'+str(f1)+'\n')
  statistics = classification_report(y_true=gold_tags, y_pred=pred_tags)
  do_file(statistics=statistics,file_t=file_t,model=models_name[model])

In [ ]:
file_t= open("tabla_results.csv","w")
file_tp= open("tabla_p.txt","w")
file_tp.write('Modelo\tPrecission\tRecall\tF1\n')
for model_name in glob.glob("/content/systemd/systemd/*"):
    print(model_name)
    if "model_" in model_name:
      try:
        do_table(model=model_name,file_t=file_t,file_tp=file_tp)
      except:
        print("NO se pudo")
file_tp.close()
file_t.close()

/content/systemd/systemd/model_12
['/content/systemd/systemd/gold/S0212-16112012000300028-1.conll', '/content/systemd/systemd/gold/S1137-66272006000300009-1.conll', '/content/systemd/systemd/gold/S1130-01082008000900015-1.conll', '/content/systemd/systemd/gold/S1139-76322013000100009-1.conll', '/content/systemd/systemd/gold/S1137-66272011000100013-2.conll', '/content/systemd/systemd/gold/S0365-66912005001200012-1.conll', '/content/systemd/systemd/gold/S1887-85712011000200007-1.conll', '/content/systemd/systemd/gold/S0365-66912012000500005-2.conll', '/content/systemd/systemd/gold/S0365-66912004000600009-2.conll', '/content/systemd/systemd/gold/S0212-71992007000200007-1.conll', '/content/systemd/systemd/gold/S0378-48352006000300005-1.conll', '/content/systemd/systemd/gold/S0004-06142006000600014-1.conll', '/content/systemd/systemd/gold/S1889-836X2016000100006-1.conll', '/content/systemd/systemd/gold/S0212-71992007000400005-1.conll', '/content/systemd/systemd/gold/S1130-01082008000800010-

/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0
0.0


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/content/systemd/systemd/model_22
['/content/systemd/systemd/gold/S0212-16112012000300028-1.conll', '/content/systemd/systemd/gold/S1137-66272006000300009-1.conll', '/content/systemd/systemd/gold/S1130-01082008000900015-1.conll', '/content/systemd/systemd/gold/S1139-76322013000100009-1.conll', '/content/systemd/systemd/gold/S1137-66272011000100013-2.conll', '/content/systemd/systemd/gold/S0365-66912005001200012-1.conll', '/content/systemd/systemd/gold/S1887-85712011000200007-1.conll', '/content/systemd/systemd/gold/S0365-66912012000500005-2.conll', '/content/systemd/systemd/gold/S0365-66912004000600009-2.conll', '/content/systemd/systemd/gold/S0212-71992007000200007-1.conll', '/content/systemd/systemd/gold/S0378-48352006000300005-1.conll', '/content/systemd/systemd/gold/S0004-06142006000600014-1.conll', '/content/systemd/systemd/gold/S1889-836X2016000100006-1.conll', '/content/systemd/systemd/gold/S0212-71992007000400005-1.conll', '/content/systemd/systemd/gold/S1130-01082008000800010-

In [ ]:
def do_file(statistics,file_t,model):
  rows_new = statistics.split('\n')
  file_t.write(model+'\n')
  for row_new in rows_new:
    for value in row_new.split('\t'):
      file_t.write(value+'\t')
    file_t.write('\n')

In [ ]:
f1 = f1_score(y_true=gold_tags, y_pred=pred_tags)
print(f1)

0.05051435590357746


In [ ]:
ii = 0
for name_coll in glob.glob("/content/systemd/systemd/model_7/test2/*"):
  if ".conll" in name_coll:
    ii+=1
print(ii)

250


In [ ]:

precision = precision_score(y_true=gold_tags, y_pred=pred_tags)
print(precision)

0.23431317128321086


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
do_file()

In [ ]:
classification_report(y_true, y_pred)

In [ ]:
y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
y_pred = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
f1_score(y_true, y_pred)

In [ ]:
def comprimir():
  fantasy_zip = zipfile.ZipFile('/content/systemd.zip', 'w')
  for folder, subfolders, filesFolder in os.walk('/content/systemd'): 
      for file in filesFolder:
          if file.endswith('.ann') or file.endswith('.txt') or file.endswith('.json'):
              fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '/content/systemd'), compress_type = zipfile.ZIP_DEFLATED)
  fantasy_zip.close()

In [ ]:
def downloadFile():
  files.download('/content/systemd.zip')

In [ ]:
comprimir()
downloadFile()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def remove_file():
  from pathlib import Path
  folder_system = "/content/systemd/systemd/compartir/gold/"
  folder = Path(folder_system)
  print(folder.iterdir())
  for file in folder.iterdir():
    try:
      file_name = file.name[0:len(file.name)-4]
      print(folder_system+file.name)
      filePath=Path(folder_system+file.name)
      filePath.unlink()
    except OSError as e:
      print(f"Error:{ e.strerror}")

In [ ]:
remove_file()